Copyright 2023 Province of British Columbia

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at 

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
# add our stuff to the path
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# other stuff
import time

# import our stuff
from importlib import reload
from src import connect, preprocess, synthetic, model

In [ ]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# where model is stored. requires credentials.txt for full path 
MODEL_BASE_PATH = 'Model/Q32'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ32RACE'
RESULTS_TABLE = 'AQ32RACE_RESULTS'

# which column to use/create 
RESPONSE_COLUMN = 'aq32race_combined'

# delimiter to send back with concatenated results
DELIMITER = 'μ' 

# threshold for accepting as a flagged category
THRESHOLD = 0.5

# upper and lower thresholds for flagging as a possible category
TENTATIVE_UPPER = 0.75
TENTATIVE_LOWER = 0.25

## Full Model Pipeline

1. Read in data from database (IDIR restricted)
2. Load in model (from LAN)
3. Preprocess data (code stored on GitHub)
    
    * Limit to cycle of interest
    * Lower case and cleaned spelling where possible
    * Turn responses into word scores (how close is each response to every word in the vocabulary code base). <br>
    <br>

4. Create predictions based on word scores
5. Re-incorprate multiple-choice responses
6. Add flag(s) for unusual model outputs
    
    * Was no category predicted
    * Are there model outputs in an 'iffy' probability range <br>
    <br>

7. Send results back to database (IDIR restricted)

In [ ]:
# FULL MODEL PIPELINE

cycle = 2

######################
#                    #
#    READ IN DATA    #
#                    #
######################
print(f'Reading in data for cycle {cycle}... ', end = '')

connection = connect.create_connection(CRED_PATH)
df_open = connect.fetch_table(RESPONSE_TABLE, connection)

print('Done.')

######################
#                    #
#    LOAD MODEL      #
#                    #
######################
print('Loading model from file... ', end = '')

clf, code_df_long = model.load_model(CRED_PATH, MODEL_BASE_PATH)
print('Done.')

######################
#                    #
#  PREPROCESS DATA   #
#                    #
######################
print('Preprocessing data... ', end = '')

# get cycle
df = df_open[df_open.cycle == cycle].reset_index(drop=True)

# clean column
df.loc[:, RESPONSE_COLUMN] = df.apply(
    lambda x: x.aq32race.lower() if x.aq32race_cleaned == None or x.aq32race_cleaned=='105' else x.aq32race_cleaned.lower(), 
    axis=1
)

# fix spelling
df.loc[:, RESPONSE_COLUMN] = df[RESPONSE_COLUMN].apply(preprocess.correct_spelling)
print('Done.')

print('Creating model inputs... ', end = '')
# inputs to model
headers = list(preprocess.get_scores('test', code_df_long, as_df = True).col_id.values)
input_df = preprocess.get_scores_from_df(df, RESPONSE_COLUMN, code_df_long, headers=headers)
print('Done.')

# outputs of model (for selected responses to be included)
print('Creating selected response outputs... ', end = '')
output_df = preprocess.get_outputs_wide(df, RESPONSE_COLUMN, code_df_long)
print('Done.')

######################
#                    #
#     RUN MODEL      #
#                    #
######################
print('Running model and extracting results... ')
results_df = model.produce_results(
    df, input_df, output_df, 
    clf,
    threshold=THRESHOLD,
    tentative_lower = TENTATIVE_LOWER,
    tentative_upper = TENTATIVE_UPPER,
    delimiter = DELIMITER
)
print('\nDone.')

In [ ]:
results_df.head()

In [ ]:
# flag those that added no model categories
results_df[results_df.n_model_categories==0]

In [ ]:
# flag those that have outputs in the 25-75% range
results_df[results_df.tentative_categories.notna()]

## Example Outputs

In [ ]:
# most common response
sentence = 'canadian'
model.list_classes(sentence, code_df_long, clf)

In [ ]:
# misspelling
sentence = 'canadien'
model.list_classes(sentence, code_df_long, clf)

In [ ]:
# multiple responses
sentence = 'black, canadian'
model.list_classes(sentence, code_df_long, clf)

In [ ]:
# multiple responses begins to become iffy if you include too many categories
sentence = 'black, canadian, white, spanish'
model.list_classes(sentence, code_df_long, clf)

In [ ]:
# doesn't confuse afro-canadian and canadian (eg)
sentence = 'afro-canadian'
model.list_classes(sentence, code_df_long, clf)